In [13]:
#%matplotlib inline
import numpy as np
import pandas as pd
import scipy.sparse
from sklearn import preprocessing
import xgboost as xgb
import pickle
#import graphviz
import matplotlib.pyplot as plt
%matplotlib inline

Read in just the testing and training data.

In [39]:
error = pd.read_pickle('scripts/cv_results/actions_e20/params1_err.p')
#error['mean'] = error.apply(np.mean, axis = 1)
# error = error.astype(np.float64)
# error['mean'] = error.mean(1)
# error['std'] = error.std(1)

In [40]:
error

,test-error-mean,test-error-std,train-error-mean,train-error-std,num_boost_rounds
0,0.163659,0.001305,0.155159,0.000223,74
1,0.163507,0.000948,0.153505,0.000129,89
2,0.163481,0.001225,0.154429,0.000137,78
3,0.163766,0.001236,0.148263,0.000219,99


In [3]:
#train_100 = pd.read_csv('Airbnb_data/train_users_2.csv', nrows = 100, index_col='id')
train_data = pd.read_csv('data/train_users_2.csv', index_col = 'id')
test_data = pd.read_csv('data/test_users.csv', index_col = 'id')
#sessions = pd.read_csv('../data/sessions.csv')
#age = pd.read_csv('Airbnb_data/age_gender_bkts.csv')
#countries = pd.read_csv('Airbnb_data/countries.csv')
all_df = pd.read_pickle('data/all_df.p')

In [163]:
for i in np.unique(test_data.first_browser):
    if i not in np.unique(train_data.first_browser):
        print i

IBrowse
Nintendo Browser
UC Browser


In [142]:
#pd.isnull(sessions.groupby('user_id')['secs_elapsed'].sum()) == False
sessions.loc[sessions.user_id == '00allnceb8',:]
#sessions.loc[sessions.user_id == 'd1mm9tcy42',:]

,user_id,action,action_type,action_detail,device_type,secs_elapsed
1150481,00allnceb8,create,submit,signup,-unknown-,NaN


In [8]:
#find most popular actions
sessions.groupby('user_id').count()

NameError: name 'sessions' is not defined

In [5]:
error = pd.read_pickle('scripts/cv_results.p')
params = pd.read_pickle('scripts/cv_params.p')

In [6]:
res = pd.concat([error, params], axis = 1)

In [7]:
res.sort_values(by = 'test-error-mean', axis = 0);

In [3]:
all_df = feakaggle_model(all_df)

#one hot encoding of features
print 'number of columns before one hot encoding', all_df.shape[1]
ohe_feats = ['gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser']
all_df = pd.get_dummies(all_df, columns = ohe_feats)
print 'number of columns after one hot encoding', all_df.shape[1]

number of columns before one hot encoding 17
number of columns after one hot encoding 161


In [6]:
with open('xgbmodels/sessions_e20_25n.p', 'rb') as f:
    bst = pickle.load(f)
features = sorted(bst.get_fscore().items(), key = lambda x: x[1], reverse = True)
[(feat, all_df.columns[int(feat[1:])], score) for feat, score in features]

NameError: name 'all_df' is not defined

In [ ]:
xgb.to_graphviz(bst, num_trees=11)

In [ ]:
submission = pd.DataFrame(np.column_stack([idx, cntr]), columns = ['id', 'country'])
submission.to_csv('submissions/datebin_model.csv', index=False)